# Projeto de Conclusão
---
### *Disclaimer:*
Este código foi elaborado para fins exclusivamente educacionais. O conteúdo aqui apresentado visa fornecer informações complementares para auxiliar o investidor na tomada de suas próprias decisões de investimento. Reforçando, nenhum tópico aqui abordado constitui qualquer tipo de indicação/oferta/solicitação de compra/venda de qualquer produto.<br/>
<br/>

---
- Primeira parte: selecionar um ativo e realizar análises quantitativas a partir dos dados de preço do mesmo

- Segunda parte: a partir da estratégia escolhida realizar o backtesting e avaliar seu desempenho

- Terceira parte: plugar e implementar sua estratégia no MetaTrader5

# 1. Instalando e importando Bibliotecas

In [ ]:
%pip install backtrader
%pip install MetaTrader5 

In [ ]:
# Análise e manipulação de dados

import numpy as np
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt

# Dados séries temporais

from datetime import datetime
import pytz
import time

# Integração com MetaTrader 5

import MetaTrader5 as mt5


In [ ]:
mt5.initialize()

# 2. Extraindo dados do MetaTrader5

In [ ]:
#Selecionar o ativo

ativo = 'WEGE3'

In [ ]:
#Selecionar timezone para São Paulo

timezone = pytz.timezone("America/Sao_Paulo")

In [ ]:
# Definir data inicial e data final, neste formato

data_inicio = datetime(2021,1,1, tzinfo=timezone)
data_fim = datetime(2022,1,1, tzinfo=timezone)

In [ ]:
#Criar um objeto chamado cotacoes com os parâmetros ativo, time frame, data inicio e data fim

cotacoes = mt5.copy_rates_range(ativo, mt5.TIMEFRAME_D1, data_inicio,data_fim)

In [ ]:
cotacoes

In [ ]:
df = pd.DataFrame(cotacoes)
df.head()

In [ ]:
# Transformar a coluna "time" para o formato de data e hora padrão

df['time']=pd.to_datetime(df['time'], unit='s')
df

# Transformar a coluna 'time' para o índice do dataframe

df.index = df['time']
df.head()

# 3. Análise Quantitativa

In [ ]:
#Vamos analisar o comportamento dos retornos da ação


retornos = pd.DataFrame(df.close.pct_change())
retornos

In [ ]:
#Visualmente vamos entender o comportamento


retornos.plot(figsize = (13,8))

In [ ]:
#Vamos realizar o cálculo da volatilidade dentro de uma janela de 10 dias e analisar como ela se comporta graficamente

janelas_vol = retornos.rolling(10).std()
janelas_vol.plot()

In [ ]:
#Agora entender como estão agrupados os retornos de acordo com a sua frequência dentro do histograma
retornos.plot.hist(bins = 80);

In [ ]:
#Observar as maiores perdas dentro de uma janela de 20 dias (aprox um mês)

dd_30 = retornos.rolling(window = 20).min()

dd_30.plot()

# 5. Backtesting

Iremos desenvolver uma estratégia de swing trade que irá comprar quando preço cruzar para cima a média móvel de 100 dias e quando cruzar novamente para baixo, iremos vender.

In [ ]:
#Verificar se não há nenhum NaN antes de entrar no backtesting de fato

df.isna().sum()

In [ ]:
#Importar os dados para dentro do backtrader

data = bt.feeds.PandasData(dataname=df)

In [ ]:
import backtrader.analyzers as btanalyzers

class rsi_media_movel(bt.Strategy):

  def __init__(self):

    self.mm_rapida = bt.indicators.SMA(self.data.close, period=100)
    self.crossup = bt.ind.CrossUp(self.data.close, self.mm_rapida)
    self.crossdown = bt.ind.CrossDown(self.data.close, self.mm_rapida)


  def next(self):
    if not self.position: #se não tiver alguma ordem aberta - condição de compra
      if self.crossup == 1:
        self.buy(size=100)

    else: #caso a ordem já estiver aberta - condição de venda
      if self.crossdown == 1:
        self.sell(size=100)
  
caixa_inicial = 30000 #definimos o montante a inicial do portfólio

cerebro = bt.Cerebro()
cerebro.addstrategy(rsi_media_movel)
cerebro.adddata(data)
cerebro.broker.setcommission(commission=0.003) #setamos a comissão da exchange
cerebro.broker.setcash(caixa_inicial)
cerebro.addanalyzer(btanalyzers.PeriodStats, _name='stats', timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
thestrats = cerebro.run()
thestrat = thestrats[0]
print('Estatísticas:', thestrat.analyzers.stats.get_analysis())
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Drawdown:', thestrat.analyzers.drawdown.get_analysis())

In [ ]:
thestrat.analyzers.stats.get_analysis()


In [ ]:
info = thestrat.analyzers.stats.get_analysis()

In [ ]:
estatisticas =info.values()
estatisticas = list(estatisticas)
estatisticas = pd.DataFrame(estatisticas, index=info.keys())
estatisticas

# 7.Automatizando o Trade

Aqui entramos na parte do MetaTrader 5, duas observações importantes:

- Primeiro que o seu ticker selecionado deve estar presente no quadro "Observação de Mercado" no canto superior direito do Metarade
- Segundo que caso você queira operar no gráfico de 1, 5, 10 ou 15 minutos você deve ir no menu superior -> Ferramentas -> Na aba Gráficos -> Selecionar a opção 'Máx. barras no gráfico" para "Unlimited", dessa maneira você conseguirá extrair e operar em um nível granular de tempo menor.


In [ ]:
#testar comunicação

import sys

count_cotacoes = 0

while count_cotacoes < 5:
    ativo = mt5.symbol_info_tick("PETR4")
    print("\r" + str(ativo.last))
    sys.stdout.flush()
    count_cotacoes += 1
    time.sleep(5)

In [ ]:
ativo.last

In [ ]:
datetime.today()

In [ ]:
# Iremos gerar um dataframe em tempo real analisando os dados diários de PETR4

symbol='PETR4'
cotacoes_ativo = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, data_inicio,datetime.today())



In [ ]:
#Transformar em um dataframe 

cotacoes_df = pd.DataFrame(cotacoes_ativo)
cotacoes_df['time']=pd.to_datetime(cotacoes_df['time'], unit='s')
cotacoes_df.index = cotacoes_df['time']

## Cálculo Média Móvel de 100 períodos
MM_periodo = 100
MMA = cotacoes_df['close'].rolling(window=MM_periodo).mean()
cotacoes_df['MMA_100p'] = MMA
cotacoes_df.tail(20)

In [ ]:
#Agora iremos desenvolver a estratégia de cruzamento de média


contador_sinais = 0
posição = 0


while (contador_sinais < 20) and (posição == 0):
    contador_sinais += 1
    print("----------------------------------------")
    print("A hora atual é", datetime.today())
    
    symbol='PETR4'
    cotacoes_ativo = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, data_inicio,datetime.today())
    cotacoes_df = pd.DataFrame(cotacoes_ativo)
    cotacoes_df['time'] = pd.to_datetime(cotacoes_df['time'], unit='s')
    cotacoes_df.index = cotacoes_df['time']
    
    MM_periodo = 100
    MMA = cotacoes_df['close'].rolling(window=MM_periodo).mean()
    cotacoes_df['MMA'] = MMA
    
    qtd = 100.0
    tick_min = mt5.symbol_info(symbol).point
    preco_compra = mt5.symbol_info_tick(symbol).ask
    preco_venda = mt5.symbol_info_tick(symbol).bid
    desvio_toler = 0
    
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": qtd,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco_compra, # preço de compra, aqui definido como ask do ativo (melhor preço de venda do book)
        "sl": preco_compra - 100 * tick_min, #stop loss (preço de entrada - 100 ticks mínimos)
        "tp": preco_compra + 100 * tick_min, #take profit (preço de entrada + 100 ticks mínimos)
        "deviation": desvio_toler, 
        "magic": 1, 
        "comment": "Trades estratégia MMA100", 
        "type_time": mt5.ORDER_TIME_DAY, 
        "type_filling": mt5.ORDER_FILLING_RETURN, 
    }
    ordem_venda = {
        "action": mt5.TRADE_ACTION_DEAL, # tipo de ordem (a mercado)
        "symbol": symbol, 
        "volume": qtd, 
        "type": mt5.ORDER_TYPE_SELL, # venda a mercado
        "price": preco_venda, # preço de compra, aqui definido como ask do ativo (melhor preço de venda do book)
        "sl": preco_venda + 100 * tick_min, #stop loss (preço de entrada - 100 ticks mínimos)
        "tp": preco_venda - 100 * tick_min, #take profit (preço de entrada + 100 ticks mínimos)
        "deviation": desvio_toler, 
        "magic": 1, 
        "comment": "Trades estratégia MMA100", 
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN, 
    }
    
    print("O preço de fechamento anterior havia sido de",cotacoes_df.close[-2],"e a MM_100p estava em",cotacoes_df.MMA[-2])
    print("O preço de fechamento atual está em",cotacoes_df.close[-1],"e a MM_100p está em",cotacoes_df.MMA[-1])
    
    if (cotacoes_df.close[-1] > cotacoes_df.MMA[-1]) and (cotacoes_df.close[-2] < cotacoes_df.MMA[-2]):
        print("Sinal de entrada: COMPRA")
        mt5.order_send(ordem_compra)
        print("COMPRA executada com sucesso!")
        print("A ordem OCO do Stoploss está no preço:",(preco_compra - 100 * tick_min))
        print("A ordem OCO do Take Profit está no preço:",(preco_compra + 100 * tick_min))
        posição = 1
        time.sleep(15)
        
    elif (cotacoes_df.close[-1] < cotacoes_df.MMA[-1]) and (cotacoes_df.close[-2] > cotacoes_df.MMA[-2]):
        print("Sinal de entrada: VENDA")
        mt5.order_send(ordem_venda)
        print("VENDA executada com sucesso!")
        print("A ordem OCO do Stoploss está no preço:",(preco_venda + 100 * tick_min))
        print("A ordem OCO do Take Profit está no preço:",((preco_venda - 100 * tick_min)))
        posição = -1
        time.sleep(15)
        
    else:
        print("Nada a fazer nesse momento. Aguarde a próxima extração em 15 seg")
        time.sleep(15)
    